In [25]:
# Importing necessary libraries and downloading NLTK data
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import SnowballStemmer

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
# Mounting Google Drive
from google.colab import drive
drive.mount("/drive", force_remount=True)

Mounted at /drive


#**Preprocessing**

Lowering, Tokenizing, stop-words removal & Stemming

In [27]:
# Function to preprocess text
def preprocess_text(text):
    words = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word.isalpha() and word not in stop_words]
    stemmer = SnowballStemmer('english')
    words = [stemmer.stem(word) for word in words]
    return words

#**scoring sentences**

In [28]:
# Function to score sentences
def score_sentences(sentences, title_words, index, cindex):
    scores = {}
    for sentence in sentences:
        words = preprocess_text(sentence)
        score = 0.0
        title_word_count = 0.0
        for word in words:
            score += index[word] / (1 + cindex[word])
            if word in title_words:
                title_word_count += 1
        if len(words) > 0:
            title_word_count = 0.1 * title_word_count / len(title_words)
            scores[sentence] = score + title_word_count
    return scores

In [29]:
# Function to generate summary
def generate_summary(text, title, max_sentence_length, threshold):
    sentences = sent_tokenize(text)
    words = preprocess_text(text)
    index = FreqDist(words)
    cindex = FreqDist(words + preprocess_text(title))
    title_words = set(preprocess_text(title))
    scores = score_sentences(sentences, title_words, index, cindex)
    summary = [sentence for sentence, score in scores.items() if score >= threshold and len(sentence) < max_sentence_length]
    return summary

#**Results**

In [30]:
# Example usage
f1 = open('/drive/My Drive/Colab Notebooks/gift-of-magi.txt')
input_text = f1.read()
input_title = "gift of magi"
max_len = 120
thresh = 2
summary = generate_summary(input_text, input_title, max_len, thresh)

In [31]:
# Print the generated summary
print("Summary of the gift-of-magi story:")
print("+++++++++++++++++++++++++++++++++++++")
print()
for sentence in summary:
    print(sentence, end=" ")

Summary of the gift-of-magi story:
+++++++++++++++++++++++++++++++++++++

ONE DOLLAR AND EIGHTY-SEVEN CENTS. AND SIXTY CENTS of it was in pennies. Three times Della counted it. One dollar and eighty-seven cents. And the next day would be Christmas. There was clearly nothing left to do but flop down on the shabby little couch and howl. Which instigates the moral reflection that life is made up of sobs, sniffles, and smiles, with sniffles predominating. While the mistress of the home is gradually subsiding from the first stage to the second, take a look at the home. A furnished flat at $8 per week. It did not exactly beggar description, but it certainly had that word on the look-out for the mendicancy squad. Also appertaining thereunto was a card bearing the name "Mr. James Dillingham Young." Della finished her cry and attended to her cheeks with the powder rag. She stood by the window and looked out dully at a grey cat walking a grey fence in a grey backyard. To-morrow would be Christma

#**Metrics**

**ROUGE**

In [32]:
pip install rouge

In [33]:
from rouge import Rouge

# Initialize Rouge
rouge = Rouge()

In [34]:
# Calculating ROUGE scores for the summary of the story

# Generated and Reference summaries.
# Generated summary is the one that is generated by our model.
# Whereas, the Reference summary is generated by ChatGPT.
generated_summary = '''
ONE DOLLAR AND EIGHTY-SEVEN CENTS. AND SIXTY CENTS of it was in pennies. Three times Della counted it. One dollar and eighty-seven cents. And the next day would be Christmas. There was clearly nothing left to do but flop down on the shabby little couch and howl. Which instigates the moral reflection that life is made up of sobs, sniffles, and smiles, with sniffles predominating. While the mistress of the home is gradually subsiding from the first stage to the second, take a look at the home. A furnished flat at $8 per week. It did not exactly beggar description, but it certainly had that word on the look-out for the mendicancy squad. Also appertaining thereunto was a card bearing the name "Mr. James Dillingham Young." Della finished her cry and attended to her cheeks with the powder rag. She stood by the window and looked out dully at a grey cat walking a grey fence in a grey backyard. To-morrow would be Christmas Day, and she had only $1.87 with which to buy Jim a present. She had been saving every penny she could for months, with this result. Twenty dollars a week doesn't go far. Only $1.87 to buy a present for Jim. Many a happy hour she had spent planning for something nice for him. Della, being slender, had mastered the art. Suddenly she whirled from the window and stood before the glass. Her eyes were shining brilliantly, but her face had lost its colour within twenty seconds. Rapidly she pulled down her hair and let it fall to its full length. Now, there were two possessions of the James Dillingham Youngs in which they both took a mighty pride. One was Jim's gold watch that had been his father's and his grandfather's. So now Della's beautiful hair fell about her, rippling and shining like a cascade of brown waters. It reached below her knee and made itself almost a garment for her. Once she faltered for a minute and stood still while a tear or two splashed on the worn red carpet. On went her old brown jacket; on went her old brown hat. Where she stopped the sign read: "Mme Sofronie. Hair Goods of All Kinds." One Eight up Della ran, and collected herself, panting. Madame, large, too white, chilly, hardly looked the "Sofronie." "I buy hair," said Madame. "Take yer hat off and let's have a sight at the looks of it." Down rippled the brown cascade. "Twenty dollars," said Madame, lifting the mass with a practised hand. "Give it to me quick" said Della. Oh, and the next two hours tripped by on rosy wings. She was ransacking the stores for Jim's present. It surely had been made for Jim and no one else. There was no other like it in any of the stores, and she had turned all of them inside out. It was even worthy of The Watch. As soon as she saw it she knew that it must be Jim's. Quietness and value--the description applied to both. Twenty-one dollars they took from her for it, and she hurried home with the 78 cents. With that chain on his watch Jim might be properly anxious about the time in any company. When Della reached home her intoxication gave way a little to prudence and reason. Which is always a tremendous task dear friends--a mammoth task. She looked at her reflection in the mirror long, carefully, and critically. what could I do with a dollar and eighty-seven cents?" At 7 o'clock the coffee was made and the frying-pan was on the back of the stove hot and ready to cook the chops. Della doubled the fob chain in her hand and sat on the corner of the table near the door that he always entered. Then she heard his step on the stair away down on the first flight, and she turned white for just a moment. The door opened and Jim stepped in and closed it. He looked thin and very serious. Poor fellow, he was only twenty-two--and to be burdened with a family! He needed a new overcoat and he was with out gloves. Jim stepped inside the door, as immovable as a setter at the scent of quail. His eyes were fixed upon Della, and there was an expression in them that she could not read, and it terrified her. It was not anger, nor surprise, nor disapproval, nor horror, nor any of the sentiments that she had been prepared for. He simply stared at her fixedly with that peculiar expression on his face. Della wriggled off the table and went for him. "Jim, darling," she cried, "don't look at me that way. I had my hair cut off and sold it because I couldn't have lived through Christmas without giving you a present. My hair grows awfully fast. Jim, and let's be happy. You don't know what a nice-what a beautiful, nice gift I've got for you." asked Jim, laboriously, as if he had not arrived at that patent fact yet, even after the hardest mental labour. "Cut it off and sold it," said Della. I'm me without my hair, ain't I?" Jim looked about the room curiously. "You say your hair is gone?" he said, with an air almost of idiocy. "You needn't look for it," said Della. "It's sold, I tell you--sold and gone, too. Shall I put the chops on, Jim?" Out of his trance Jim seemed quickly to wake. For ten seconds let us regard with discreet scrutiny some inconsequential object in the other direction. Eight dollars a week or a million a year--what is the difference? A mathematician or a wit would give you the wrong answer. The magi brought valuable gifts, but that was not among them. I his dark assertion will be illuminated later on. Jim drew a package from his overcoat pocket and threw it upon the table. "Don't make any mistake, Dell," he said, "about me. But if you'll unwrap that package you may see why you had me going a while at first." White fingers and nimble tore at the string and paper. And then an ecstatic scream of joy; and then, alas! For there lay The Combs--the set of combs, side and back, that Della had worshipped for long in a Broadway window. Beautiful combs, pure tortoise-shell, with jewelled rims--just the shade to wear in the beautiful vanished hair. And now, they were hers, but the tresses that should have adorned the coveted adornments were gone. And then Della leaped up like a little singed cat and cried, "Oh, oh!" Jim had not yet seen his beautiful present. She held it out to him eagerly upon her open palm. The dull precious metal seemed to flash with a reflection of her bright and ardent spirit. You'll have to look at the time a hundred times a day now. I want to see how it looks on it." Instead of obeying, Jim tumbled down on the couch and put his hands under the back of his head and smiled. "Dell," said he, "let's put our Christmas presents away and keep 'em a while. They're too nice to use just at present. I sold the watch to get the money to buy your combs. And now suppose you put the chops on." The magi, as you know, were wise men--wonderfully wise men-who brought gifts to the Babe in the manger. They invented the art of giving Christmas presents. Being wise, their gifts were no doubt wise ones, possibly bearing the privilege of exchange in case of duplication. But in a last word to the wise of these days let it be said that of all who give gifts these two were the wisest. Of all who give and receive gifts, such as they are wisest.
'''
reference_summary = '''
Della, with only $1.87, frets over affording a Christmas gift for her husband, Jim. Despite careful saving, her funds fall short for a present worthy of him. In a bold move, she sells her beautiful hair for $20 to buy him a platinum fob chain for his watch. Her sacrifice highlights her deep love for Jim. Unbeknownst to her, Jim sells his watch to purchase expensive combs for her hair, rendering her gift useless. Both gifts, bought with selfless sacrifices, reflect the magi's wisdom in the biblical narrative. Their actions symbolize the true spirit of giving and selflessness during Christmas, surpassing material worth with heartfelt sacrifices made out of love and devotion. The story emphasizes the priceless value of love and sacrifices over material possessions, echoing the essence of the holiday season.
'''

# Compute ROUGE scores
scores = rouge.get_scores(generated_summary, reference_summary)

# Access specific ROUGE scores
rouge_1_scores = scores[0]['rouge-1']
rouge_2_scores = scores[0]['rouge-2']
rouge_l_scores = scores[0]['rouge-l']

# Print ROUGE scores
print("ROUGE-1 scores:", rouge_1_scores)
print("ROUGE-2 scores:", rouge_2_scores)
print("ROUGE-L scores:", rouge_l_scores)

ROUGE-1 scores: {'r': 0.40425531914893614, 'p': 0.060995184590690206, 'f': 0.10599720832143855}
ROUGE-2 scores: {'r': 0.10852713178294573, 'p': 0.011627906976744186, 'f': 0.021005249564649376}
ROUGE-L scores: {'r': 0.39361702127659576, 'p': 0.0593900481540931, 'f': 0.10320780804249853}


**BLEU**

In [35]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

# Reference and candidate sentences
reference_summary = "Della, with only $1.87, frets over affording a Christmas gift for her husband, Jim. Despite careful saving, her funds fall short for a present worthy of him. In a bold move, she sells her beautiful hair for $20 to buy him a platinum fob chain for his watch. Her sacrifice highlights her deep love for Jim. Unbeknownst to her, Jim sells his watch to purchase expensive combs for her hair, rendering her gift useless. Both gifts, bought with selfless sacrifices, reflect the magi's wisdom in the biblical narrative. Their actions symbolize the true spirit of giving and selflessness during Christmas, surpassing material worth with heartfelt sacrifices made out of love and devotion. The story emphasizes the priceless value of love and sacrifices over material possessions, echoing the essence of the holiday season."

generated_summary = "ONE DOLLAR AND EIGHTY-SEVEN CENTS. AND SIXTY CENTS of it was in pennies. Three times Della counted it. One dollar and eighty-seven cents. And the next day would be Christmas. There was clearly nothing left to do but flop down on the shabby little couch and howl. Which instigates the moral reflection that life is made up of sobs, sniffles, and smiles, with sniffles predominating. While the mistress of the home is gradually subsiding from the first stage to the second, take a look at the home..."

# Split reference summary into sentences
reference_sentences = nltk.sent_tokenize(reference_summary)

# Calculate BLEU score for individual sentences
individual_bleu_scores = [sentence_bleu([ref], generated_summary) for ref in reference_sentences]

# Calculate corpus BLEU score
corpus_bleu_score = corpus_bleu([reference_sentences], [generated_summary])

print("Individual BLEU scores for each reference sentence:")
print(individual_bleu_scores)
print("\nCorpus BLEU score:")
print(corpus_bleu_score)


Individual BLEU scores for each reference sentence:
[0.07848939586655637, 0.038175358879525445, 0.05725018372138839, 0.019370888541844284, 0.06304140786959997, 0.09748090348910751, 0.15925261684393688, 0.1183364596447706]

Corpus BLEU score:
0.2862652777808761
